# Validation of Non-Cooperative Reader Protein Binding and Unbinding

In this notebook, we validate the binding and unbinding of a reader protein, heterochromatin protein 1 (HP1), to a chromatin fiber modified by H3K9me3.

### Import Modules and Navitate to Root Directory

In [ ]:
# Built-in Modules
import os
import sys
from multiprocessing import Pool

cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)
os.chdir("../..")

# External Modules
import numpy as np

In [ ]:
# Custom Modules
import chromo
import chromo.mc as mc
from chromo.polymers import Chromatin
import chromo.binders
from chromo.fields import UniformDensityField
import chromo.mc.mc_controller as ctrl
from chromo.util.reproducibility import get_unique_subfolder_name

### Define Confinement and Polymer Parameters

In [ ]:
# Confine to spherical chrom. territory 1800 um diameter (Cremer & Cremer 2001)
confine_type = "Spherical"
confine_length = 900

# Specify polymer
num_beads = 25000
bead_spacing = 16.5

# Scale down the confinement so the density matches that of a chromosome
# inside a chromosome territory
frac_full_chromo = num_beads / 393216
confine_length *= np.cbrt(frac_full_chromo)

# Specify chemical modifications
chem_mods_path = np.array(
    ["chromo/chemical_mods/meth"]
)
chemical_mods_all = Chromatin.load_seqs(chem_mods_path)

# Specify how many repeated simulations to evaluate at each chemical potential
chemical_potentials = np.linspace(-4.5, 1.5, 13)
num_samples = 3

### Simulate Non-cooperative Reader Protein Binding at Variable Chemical Potential

In [ ]:
for i in range(num_samples):

    def run_sim(chemical_potential):
        """Run a simulation.

        Parameters
        ----------
        chemical_potential : float
            Chemical potential of the reader protein

        Notes
        -----
        This function is defined to enable parallel processing.
        """

        # Specify reader proteins
        binders = [chromo.binders.get_by_name('HP1')]
        binders[0].chemical_potential = chemical_potential

        # Turn off interactions between reader proteins
        binders[0].interaction_energy = 0.0

        # Reformat reader proteins into a dataframe format
        binders = chromo.binders.make_binder_collection(binders)

        # Pick a random segment of the chromosome to simulate
        start_ind = np.random.randint(len(chemical_mods_all))
        chemical_mods = np.take(chemical_mods_all, np.arange(start_ind, start_ind + num_beads), mode="wrap", axis=0)

        # Define the polymer
        p = Chromatin.confined_gaussian_walk(
            'Chr-1',
            num_beads,
            bead_spacing,
            states=chemical_mods.copy(),
            confine_type=confine_type,
            confine_length=confine_length,
            binder_names=np.array(['HP1']),
            chemical_mods=chemical_mods,
            chemical_mod_names=np.array(['H3K9me3'])
        )

        # Specify the field containing the polymers
        n_bins_x = int(round(63 * np.cbrt(frac_full_chromo)))
        x_width = 2 * confine_length
        n_bins_y = n_bins_x
        y_width = x_width
        n_bins_z = n_bins_x
        z_width = x_width
        udf = UniformDensityField(
            [p], binders, x_width, n_bins_x, y_width,
            n_bins_y, z_width, n_bins_z, confine_type=confine_type,
            confine_length=confine_length, chi=1, fast_field=1
        )

        # Specify the bead selection and move amplitude bounds
        polymers = [p]
        amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds(polymers)

        # Run the simulation
        num_snapshots = 200
        mc_steps_per_snapshot = 50000
        mc.polymer_in_field(
            [p],
            binders,
            udf,
            mc_steps_per_snapshot,
            num_snapshots,
            amp_bead_bounds,
            amp_move_bounds,
            output_dir='output'
        )

    pool = Pool(len(chemical_potentials))
    pool.map(run_sim, chemical_potentials)